In [13]:
from Utilities import Mytokenizer
from torch.utils.data import Dataset
import torch

import Transformer

In [14]:
train_path = './train.txt'
data_path = './cmn.txt'
tokenizer = Mytokenizer(data_path,'en')

中文字典字数 3643
英文字典字数 8349


In [15]:
class Mydataset(Dataset):
    """
    @file_path 数据存储位置
    @tokenizer 将文字id化的实例化后的Tokenizer
    @文件中 trg数据的位置
    
    由于Decoder的输入要求，一句话应被切分成多段，目标词数有多少就应切分多少次
    那么用一个自己写在Transformer中的batch类封装一句话，会自动的给句子mask，
    并且切分Decoder的输入输出,但是注意例如:
    trg为 "I love food" 那么应该输入四次，输出四次
        输入                   输出
    <BOS> mask mask mask   I 
    <BOS> I    mask mask   I love
    <BOS> I    love mask   I love food
    <BOS> I    love food   I love food <EOS>
    
    文件中的样子如下，中译英任务英文为目标语言 trg_index应为0
    Hi	嗨
    Hi	你 好
    Run	你 用 跑 的
    """
    def __init__(self,file_path,tokenizer,trg_index=0):
        self.tokenizer = tokenizer
        #读取所有文本
        with open(file_path,'r',encoding='utf8') as f:
            self.lines = f.readlines()
        #self.trg_count_words_line = []
        self.length = len(self.lines)
        self.trg_index = trg_index
        if trg_index==0:
            self.src_index = 1
        else:
            self.src_index = 0
        
    def __getitem__(self,index):
        src = self.lines[index].split('\t')[self.src_index]
        src = src.split('\n')[0]
        print("src:",src)
        trg = self.lines[index].split('\t')[self.trg_index]
        print("trg:",trg)
        #如上面的例子 三个词的句子应有四个样本,所以应该拷贝三次
        copy_time = len(trg.split(' '))
        print(copy_time)
        # src_id化 这里简单定义了src使用中文
        src_id = self.tokenizer.ch_token_id([src],len(src.split(' ')))
        print("src_id:",src_id)
        trg_id = self.tokenizer.en_token_id([trg],len(trg.split(' '))+2)
        print("trg_id:",trg_id)
        src_tensor = torch.LongTensor(src_id)
        trg_tensor = torch.LongTensor(trg_id)
        print('src_tensor:',src_tensor.shape,'trg_tensor:',trg_tensor.shape)
        #复制   
        #src_tensor = src_tensor.repeat(copy_time+1,1)
        #trg_tensor = trg_tensor.repeat(copy_time+1,1)
        #print(src_tensor)
        #print(trg_tensor)
        b = Transformer.Batch(src_tensor,trg_tensor)
        print('数据最终形态')
        print('输入',b.trg)
        print('输出',b.trg_y)
        print('mask',b.trg_mask)
        return b
    def __len__(self):
        return self.length
    
        

In [16]:
dataset = Mydataset(train_path,tokenizer)
print(len(dataset))
b1 = dataset.__getitem__(1024)

23635
src: 我 真 蠢
trg: I'm so stupid
3
src_id: [[16, 226, 309]]
trg_id: [[1, 23, 175, 698, 2]]
src_tensor: torch.Size([1, 3]) trg_tensor: torch.Size([1, 5])
数据最终形态
输入 tensor([[  1,  23, 175, 698]])
输出 tensor([[ 23, 175, 698,   2]])
mask tensor([[[ True, False, False, False],
         [ True,  True, False, False],
         [ True,  True,  True, False],
         [ True,  True,  True,  True]]])


将要测试输入输出的模块 实例化步骤

In [19]:
src_vocab,trg_vocab = tokenizer.get_vocab()
d_model = 512
#这是一个简单的Transformer网络，只有一层encoder decoder 注：无生成器
model = Transformer.make_model(src_vocab,trg_vocab,1,d_model)
en_embedding = Transformer.Embeddings(d_model,trg_vocab)
ch_embedding = Transformer.Embeddings(d_model,src_vocab)
multihead_attention = Transformer.MultiHeadedAttention(8,d_model)
pe = Transformer.PositionalEncoding(d_model,0.1,max_len=20)
generater = Transformer.Generator(d_model,trg_vocab)

C:\Users\YBC\jupyter\Transformer.py:398: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  nn.init.xavier_uniform(p)


## 数据模拟
通过未训练的网络运行数据获得对应的数据形状\
顺序为Embedding->PositionEncoding->MultiheadAttention\
Encoder-MultiHead 输入来源为q:src_tensor k:src_tensor v:src_tensor\
Dncoder-MultiHead\
输入来源为\
1&emsp;q:trg_tensor&emsp;k:trg_tensor&emsp;&emsp;&emsp;v:trg_tensor&emsp;mask:trg_mask\
2&emsp;q:trg_tensor&emsp;k:encoder_output&emsp;v:encoder_output&emsp;mask:src_mask


In [35]:
#这里是模拟一个样本输入Transformer内部数据处理流程，并且打印从各个模块出来的数据形状
src_tensor = ch_embedding(b1.src)
trg_tensor = en_embedding(b1.trg)
print("Embedding：",src_tensor.shape,trg_tensor.shape)
src_tensor = pe(src_tensor)
trg_tensor = pe(trg_tensor)
print("PositionEncoding:",src_tensor.shape,trg_tensor.shape)
print(b1.src_mask.shape)
encoder_output = multihead_attention(src_tensor,src_tensor,src_tensor,b1.src_mask)
print('Encoder-Multihead:',encoder_output.shape)
decoder_output = multihead_attention(trg_tensor,trg_tensor,trg_tensor,b1.trg_mask)
decoder_output = multihead_attention(trg_tensor,encoder_output,encoder_output,b1.src_mask)
print('Decoder-masked-Multihead:',decoder_output.shape)
print("这里开始完整Transformer结构数据流程")
#这里是直接将数据输入Transformer 与上面代码无关
trg_input = b1.trg
copy_time = trg_input.shape[1]
#print(trg_input.shape)
trg_input = trg_input.repeat(copy_time,1)
src_input = b1.src
src_input = src_input.repeat(copy_time,1)
print('输入模型的数据为(其实就是一句话复制了四遍):\n',src_input,'\n',trg_input)
transformer_output = model(src_input,trg_input,b1.src_mask,b1.trg_mask)
print("Total Model:",transformer_output.shape)
generater_output = generater(transformer_output)
print("Generater:",generater_output.shape)
print("目前我们已经获得了四句话，四个词的id了,当然这四句应该是")
print("I'm |<PAD>|<PAD>|<PAD>")
print("I'm |so | <PAD> |<PAD>")
print("I'm |so |stupid |<PAD>")
print("I'm |so |stupid |<EOS>")
print("那么我们需要将这四句话的id化的向量与generator的输出计算损失")
print("但这里需要注意，每次只计算一个词的损失，如第一行应计算I'm\n\
第二行应计算so 以此类推 其他词的差距不进行计算")

Embedding： torch.Size([1, 3, 512]) torch.Size([1, 4, 512])
PositionEncoding: torch.Size([1, 3, 512]) torch.Size([1, 4, 512])
torch.Size([1, 1, 3])
Encoder-Multihead: torch.Size([1, 3, 512])
Decoder-masked-Multihead: torch.Size([1, 4, 512])
这里开始完整Transformer结构数据流程
输入模型的数据为(其实就是一句话复制了四遍):
 tensor([[ 16, 226, 309],
        [ 16, 226, 309],
        [ 16, 226, 309],
        [ 16, 226, 309]]) 
 tensor([[  1,  23, 175, 698],
        [  1,  23, 175, 698],
        [  1,  23, 175, 698],
        [  1,  23, 175, 698]])
Total Model: torch.Size([4, 4, 512])
Generater: torch.Size([4, 4, 8349])
目前我们已经获得了四句话，四个词的id了,当然这四句应该是
I'm |<PAD>|<PAD>|<PAD>
I'm |so | <PAD> |<PAD>
I'm |so |stupid |<PAD>
I'm |so |stupid |<EOS>
那么我们需要将这四句话的id化的向量与generator的输出计算损失
但这里需要注意，每次只计算一个词的损失，如第一行应计算I'm
第二行应计算so 以此类推 其他词的差距不进行计算


## mask 展示
下面展示以下mask后的decoder输入
这个操作其实是在attention中执行的，这里只是简单展示

In [32]:
trg_input.masked_fill(b1.trg_mask==0,-1e9)

tensor([[[          1, -1000000000, -1000000000, -1000000000],
         [          1,          23, -1000000000, -1000000000],
         [          1,          23,         175, -1000000000],
         [          1,          23,         175,         698]]])

## 以上为一条数据全部流程
这里没有演示损失函数计算以及优化过程，损失函数哈佛学习版使用了KL散度，而attention论文中使用了交叉熵，这里不做评价，请自行选择合适的损失函数\
Transformer的优化是带有热身的，1个小时连热身都跑不完，所以就算了\
一条数据的如何处理如何走过整个模型的样子已经演示了\
相信你肯定会批量训练了吧🤡
